# Supervised Learning

To recap, just as with logistic regression, we will be analzing data coming from the US Bureau of Transportation Statistics, who recorded (a lot of) data about flights in the US from 1987 to 2008 to investigate the causes of delays.

To limit this data set, we will only consider data from 2008 and selected around 100,000 instances. In cleaning the data we also removed some of the columns:

* We removed non-ordinal data
* We removed data that can only be known when the plane has already arrived

The aim of the task is to build a classifier that can predict whether a flight will arrive with a major delay, given the parameters at takeoff. In doing so, we will run through two discriminative classifiers, namely, k-Nearest Neighbours and Decision Trees.

To aim visualisation and get a sense of how the models work, we will focus on classifying major delays using only two predictor variables (Distance and Departure delay)

### Loading the data

As usual, let's start by loading in some essential libraries: 

* Import `pandas`, `numpy`, `matplotlib` and `seaborn` 
* Load the data corresponding to the file `flights08_cleaned.csv`
* From this data, create a `major_delay` variable corresponding to the `MajorDelay` column 
* Finally, filter the data down to the columns `Distance` and `DepDelay`, call this `data_partial`

In [ ]:
# Code to load the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [ ]:
# Add your code to load the data and create a major_delay variable
data_partial = ...
data = pd.read_csv("data/flights08_clean.csv")

major_delay = data["MajorDelay"]

data_partial = data[["Distance", "DepDelay"]]


### Inspect the data

Before diving into any classification, feel free to familiarise yourself with the data set once more. In particular, consider using the following commands:

* Do the attributes make sense? (see [here](http://stat-computing.org/dataexpo/2009/the-data.html) if you need any clarification)
* What's the shape of the dataset? Consider `data_partial.shape`
* How many unique values are present per attribute? Consider `data_partial.apply(pd.Series.nunique)` 
* What do the distributions of the variables look like?

In [ ]:
# Add your code here to do a first exploration of the data
print("Shape of the data: {}\n".format(data_partial.shape))

print("\nNumber of unique values?...\n")
print(data_partial.nunique())

plt.figure(figsize=(8, 6))
sns.distplot(data_partial["DepDelay"])


### Splitting the data into train and test sets

Before fitting any models, it's important to split our data into a train and test set. As ever, sklearn has your back.

Import the function `train_test_split` from `sklearn.model_selection` and check the documentation using the `?` (note that its always good practice to check the documentation!)

In [ ]:
# Add your code to load the function and check the docs
from sklearn.model_selection import train_test_split
?train_test_split


The key options that you are most likely to use are:

* `test_size` a proportion so a number between 0 and 1, typically `0.2` or `0.3`
* `random_state` an arbitrary integer to seed the train-test split so that your experiments are reproducible
* `stratify` in the case of an imbalanced data set, you want to make sure your test set and your training set contain similar proportions of the different classes (where the classes are defined by our `major_delay` outcome variable). 

Create `X_train_p`, `X_test_p`, `y_train_p`, `y_test_p` out of `data_partial`, use `0.3` as proportion for test and set the random state to `101`. Specify `major_delay` as the stratifier. 

In [ ]:
# Add your code here
X_train_p, X_test_p, y_train_p, y_test_p = train_test_split(data_partial, major_delay, 
                                                            test_size=0.3, random_state=101,
                                                            stratify=major_delay)


## The kNN model

As we have seen, sklearn models are really easy to use. For supervised models, all we need to do is:

* Import the relevant class (here `KNeighborsClassifier` from `sklearn.neighbors`)
* Generate an instance of the class specifying parameters (let's start by setting `k`, the number of neighbors, to 5)
* Apply the `fit` method of the instance on the training data
* Apply the `predict` method of the instance on the training data (and ultimately on the testing data)


In [ ]:
# Import the class and create an instance where k (n_neighbours) equals 5
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# now fit it to the training data
knn.fit(X_train_p, y_train_p)


### Visualising Decision Boundaries

You may have noticed that the fitting of the KNN model was instantaneous. This is because there is effectively no "learning" in the KNN model, all the computations are done at prediction time. So let's investigate the decision boundary and, in doing so, make some predictions.

Inspecting the decision boundaries between our classes of interest is a good way to understand both how the KNN algorithm works and checking whether or not we have overlooked anything in our data analysis (such as whether or not our data is appropriately formatted).

Displaying boundaries can be a bit involved, but is very worthwhile. In the code below we investigate our two features (`Distance` and `DepDelay`) over a grid of possible values and classify every point in the grid. Note that, in this example, we only have two features so can plot a 2D grid: in a more typical classification problem we would have many more features which would be much harder to visualise in a meaningful way.

Let's do this for our two features:
* Create a meshgrid of possible values for our features (using `np.meshgrid`), say, $100\times100$
* Then, for every point in the grid, use the KNN model to classify each point and store in the vector `Z`
* Reshape `Z` so that is has the shape of the grid
* Use `pcolormesh` to display the result of the classification

[This tutorial](http://scikit-learn.org/stable/auto_examples/neighbors/plot_nearest_centroid.html) does something fairly similar and may prove helpful if you need further clarification about this process (sklearn has some very good tutorials!)

In [ ]:
# First, we find the minimum and maximum values for our features
x_min, x_max = 0, data_partial['Distance'].max()
y_min, y_max = data_partial['DepDelay'].min(), data_partial['DepDelay'].max()

# Then create a meshgrid of 100 evenly spaced values over these ranges
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

# Create our predicted class vector, Z. Note that we reshape this later to match the meshgrid's shape
Z = np.zeros(100*100)

# To facilitate iterating over the meshgrid, flatten xx and yy into vectors 
xxr = xx.ravel()
yyr = yy.ravel()

# Then iterate over each cell in the meshgrid and classify the values of Distance and CRSDepTime
for i in range(len(Z)):
    Z[i] = knn.predict(np.array([xxr[i], yyr[i]]).reshape(1, -1))[0]

# Reshape the vector of predicted values so that it has the same shape as the meshgrid
Z = Z.reshape(xx.shape)

# Finally, display the results using pcolormesh
plt.figure(figsize=(8, 6))
cmap = plt.get_cmap('Blues', 2)
plt.pcolormesh(xx, yy, Z, cmap=cmap)
plt.colorbar(ticks=[0, 1])
plt.xlabel("Distance")
plt.ylabel("DepDelay")

### Tuning the k Parameter

As the plot above indicates, our model with `k` set to 5 creates a fairly linear decision boundary: flights delayed by approximately 50 minutes or more are classified as major delays and while there is some variation over flight distance, this effect is muted.

However, this boundary is highly dependent on the parameter `k` which represents the number of nearest training samples that are used to classify an unlabeled data point.

A key part of the k-Nearest Neighborhood algorithm is the choice of k. So far, we have been using a value of 5. In this case, each new data point is predicted by the labels of its nearest 5 neighbours in the training set. However, `k` is a parameter which can and should be tuned. It can take any value from 1 to the number of data points in the training set.

To demonstrate this, let's consider what happens to the decision boundary as we increase the number of neighbours we use to predict class labels for new data points. Let's try four different values of `k`:
* Create a list of `k` values to investigate, for example: `parameters = [5, 50, 100, 500]`
* Write a for loop over this list which instantiates the KNN object with the k parameter
* Fit this KNN object with our training data
* Call `plot_function` which will plot a decision boundary as in the previous code block

In [ ]:
def plot_function(knn_model):
    # As before, identify the range of our two features
    x_min, x_max = 0, data_partial['Distance'].max()
    y_min, y_max = data_partial['DepDelay'].min(), data_partial['DepDelay'].max()

    # Create a meshgrid of 100 evenly spaced values over these ranges
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    # Use the passed KNN object to predict values
    Z = knn_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape).astype(float)
    
    # Finally, display the results using pcolormesh
    plt.figure(figsize=(8, 6))
    cmap = plt.get_cmap('Blues', 2)
    plt.pcolormesh(xx, yy, Z, cmap=cmap)
    plt.colorbar(ticks=[0, 1])
    plt.xlabel("Distance")
    plt.ylabel("DepDelay")
    plt.show()

    
# Create a list of k values to investigate and then call plot_function on the fitted object
parameters = [5, 50, 100, 500]

for k in parameters:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_p,y_train_p)
    plot_function(knn)


### Training and Testing Accuracy over k

As you can see, the decision boundary can be highly sensitive to the `k` parameter. This boundary is essential to the performance of the overall model. So far, however, we have only considered a simplified, two-feature model. To get a better sense of how the classifer's performance is influenced by the choice of the `k` parameter, we need to train our model on the full data set.

Let's reload the original data and train a full model to investigate how training and testing accuracy is influenced by our choice of `k`:

* From our `data` dataframe, drop the `MajorDelay` column as we already have this information in the `major_delay` variable
* Create `X_train`, `X_test`, `y_train`, `y_test` out of `data` and `major_delay` from the sklearn `train_test_split` function. Again, use `0.3` as proportion for test and set the random state to `101`. Specify `major_delay` as the stratifier

In [ ]:
# Reload the data file, drop the MajorDelay variable, and create training and test data sets
major_delay = data["MajorDelay"]

X_train, X_test, y_train, y_test = train_test_split(data.drop('MajorDelay', axis=1), major_delay, 
                                                    test_size=0.3, random_state=101,
                                                    stratify=major_delay)


After running these preparatory steps, we can then iterate over a number of values of `k`, training and evaluating KNNs at each iteration. To do so, we need to create variables to store our parameters of `k`, the classifier's training performance, and the classifier's test performance:

* Create a list of `k` values to investigate, for example: `parameters = [5, 25, 50, 100]`
* Then create two empty lists, `train_accuracy` and `test_accuracy`
* Write a for loop over `parameters` which instantiates the KNN object with the `k` parameter
* Fit this KNN object with our training data and, using the `score()` method append the training accuracy and the test accuracy to their respective lists
* Finally, call the `plot_train_test_accuracy` function with arguments `parameters`, `training_accuracy`, and `test_accuracy`

In [ ]:
def plot_train_test_accuracy(k, train_accuracy, test_accuracy):
    plt.figure(figsize=(8, 6))
    
    plt.plot(k, train_accuracy, linewidth=2.0, color='r', label="Train Accuracy")
    plt.plot(k, test_accuracy, linewidth=2.0, color='b', label="Test Accuracy")
    
    plt.xlabel("k")
    plt.ylabel("Accuracy")
    plt.legend(loc='upper right')
    plt.show()

# + Create two empty lists for train and test accuracy then iterate over a list of k values
train_accuracy = []
test_accuracy = []

parameters = [1, 2, 3, 4, 5, 25, 50, 100]

for k in parameters:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train,y_train)
    train_accuracy.append(knn.score(X_train,y_train))
    test_accuracy.append(knn.score(X_test,y_test))
    
plot_train_test_accuracy(parameters, train_accuracy, test_accuracy)


### Bias and Variance

In this case, our KNN demonstrates good training performance across values of `k`, up to around `1000`. Importantly, the model's accuracy is similar across both training and test data sets. This is a good indication that the model has not overfitted on the training set: if it were, we would expect a substantial drop in performance on the test set as the model failed to generalise to unseen data points. 

Considering the plot above, and the decision boundaries that we generated earlier, it is worth discussing model complexity. Increasing `k` results in averaging more nearest neighbours to label unseen data points, which results in smoother decision boundaries (compare $k=5$ to $k=500$). With $k=1$, however, the separation between our two classes is jagged and islands appear. 

The value of `k` indicates the degree of model complexity, and relates to the concepts of bias and variance. While bias arises from erroneous assumptions in the model's specification, variance arises from high sensitivity to variations in the training set.

When `k` is equal to `1`, our model is most complex. At this point, are we observing high variance or high bias? What happens to the bias as we increase `k`?

### Limitations

Before investigating other classifiers, it is worth considering some of the shortcomings of our data set. For instance, we included in our model the days of the week (`DayOfWeek`). When handling this data, the KNN treats Mondays (`1`) as maximally different to Sundays (`7`). Why might this be a problem? Can you identify any other features which might have similar shortcomings?

## Decision Tree Classifier (DTC)

Just as with KNN, sklearn provides an easy, highly optimised implementation for DTCs. A DTC has a hierarchical structure, where each node corresponds to a feature and a split. To get started with DTCs, let's build a simple model with a maximum depth (i.e. number of splits) of `2`:

* Import the relevant class (here `DecisionTreeClassifier` from `sklearn.tree`)
* Inspect the documentation using `?`
* Instantiate the `DecisionTreeClassifer` object with `max_depth` set to `2`
* Fit the object on `X_train` and `y_train`

In [ ]:
# Add your code to import the object and check the docs
from sklearn.tree import DecisionTreeClassifier
?DecisionTreeClassifier

# Create an instance where max_depth equals 2 and fit it to the training data
dtc = DecisionTreeClassifier(max_depth=3)

dtc.fit(X_train, y_train)


### Visualising the DTC

Using the `graphviz` package, we can easily visualise the tree structure in the notebook. To do so execute the cell below. For more information, see the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html)

In [ ]:
from sklearn.tree import export_graphviz
import graphviz
dot_data = export_graphviz(dtc, out_file=None, 
    feature_names=X_train.columns,  
    class_names=['not delayed', 'delayed'],  
    filled=True, rounded=True,  
    special_characters=True, rotate=True)
graph = graphviz.Source(dot_data)
graph

### Visualising Decision Boundaries

As in the case for KNNs, let's visualise the decision boundary using two of the features in our data set (again, we use `Distance` and `DepDelay`).

Since we have the variables already loaded, we can use `X_train_p`, `y_train_p`, etc. Start by fitting a more complex tree with at least 5 levels, then:

* Create a meshgrid of possible values for our features (using `np.meshgrid`), say, $100\times100$
* Then, for every point in the grid, use the DTC model to classify each point and store in the vector `Z`
* Reshape `Z` so that is has the shape of the grid
* Use `pcolormesh` to display the result of the classification

Can you interpret the decision boundary?

In [ ]:
# First, create a DTC with max_depth equal to 5 and fit it:
dtc = DecisionTreeClassifier(max_depth=5)
dtc.fit(X_train_p, y_train_p)

# Second, we find the minimum and maximum values for our features
x_min, x_max = 0, data['Distance'].max()
y_min, y_max = data['DepDelay'].min(), data['DepDelay'].max()

# Then create a meshgrid of 100 evenly spaced values over these ranges
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                     np.linspace(y_min, y_max, 100))

# Create our predicted class vector, Z. Note that we reshape this later to match the meshgrid's shape
Z = np.zeros(100*100)

# To facilitate iterating over the meshgrid, flatten xx and yy into vectors 
xxr = xx.ravel()
yyr = yy.ravel()

# Then iterate over each cell in the meshgrid and classify the values of Distance and CRSDepTime
dtc.predict(np.array([xxr[0], yyr[0]]).reshape(1, -1))[0]

for i in range(len(Z)):
    Z[i] = dtc.predict(np.array([xxr[i], yyr[i]]).reshape(1, -1))[0]

# Reshape the vector of predicted values so that it has the same shape as the meshgrid
Z = Z.reshape(xx.shape)

# Finally, display the results using pcolormesh
plt.figure(figsize=(8, 6))
cmap = plt.get_cmap('Blues', 2)
plt.pcolormesh(xx, yy, Z, cmap=cmap)
plt.colorbar(ticks=[0, 1])
plt.xlabel("Distance")
plt.ylabel("DepDelay")

### Tuning DTC depth

In a DTC the decision boundary is a visual representation of the hierarchical variable thresholds explicitly described by the nodes of the tree itself.

However, this boundary is sensitive to the depth of the tree. To demonstrate this, let's consider what happens to the decision boundary as we inspect the decision boundary over a range of `max_depth` values:

* Create a list of `max_depth` to investigate, for example: `depths = [2, 5, 10, 50, 100]
* Write a for loop over this list which instantiates the DTC object with the `max_depth` parameter
* Fit this DTC object with our partial training data
* Call `plot_function` which will plot a decision boundary 

In [ ]:
def plot_function(dtc_model):
    # As before, identify the range of our two features
    x_min, x_max = 0, data_partial['Distance'].max()
    y_min, y_max = data_partial['DepDelay'].min(), data_partial['DepDelay'].max()

    # Create a meshgrid of 100 evenly spaced values over these ranges
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    # Use the passed DTC object to predict values
    Z = dtc_model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape).astype(float)
    
    # Finally, display the results using pcolormesh
    plt.figure(figsize=(8, 6))
    cmap = plt.get_cmap('Blues', 2)
    plt.pcolormesh(xx, yy, Z, cmap=cmap)
    plt.colorbar(ticks=[0, 1])
    plt.xlabel("Distance")
    plt.ylabel("DepDelay")
    plt.show()

    
# Create a list of depth values to investigate and then call plot_function on the fitted object
depths = [5, 10, 50, 100]

for d in depths:
    dtc = DecisionTreeClassifier(max_depth=d)
    dtc.fit(X_train_p, y_train_p)
    plot_function(dtc)


### Training and Testing Accuracy over DTC depth

As you can see, the decision boundary can be quite sensitive to the `max_depth` parameter. While this boundary is essential to the predictions of the model, we have only considered a simple, two-feature model.

To get a better sense of how the classifer's performance is influenced by the choice of the `max_depth` parameter, we need to train our model on the full data set:

* Create a list of `max_depth` values to investigate, for example: `depths = [1, 2, 5, 10, 50, 100]`
* Then create two empty lists, `train_accuracy` and `test_accuracy`
* Write a for loop over `parameters` which instantiates the DTC object with the `max_depth` parameter
* Fit this DTC object with our training data and, using the `score()` method append the training accuracy and the test accuracy to their respective lists
* Finally, call the `plot_train_test_accuracy` function with arguments `depths`, `training_accuracy`, and `test_accuracy`

(**Bonus**) if you have time, try to explore the parameters of the DTC. What do they mean? Do they help? See also the [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier). 

In [ ]:
def plot_train_test_accuracy(depth, train_accuracy, test_accuracy):
    plt.figure(figsize=(8, 6))
    
    plt.plot(depth, train_accuracy, linewidth=2.0, color='r', label="Train Accuracy")
    plt.plot(depth, test_accuracy, linewidth=2.0, color='b', label="Test Accuracy")
    
    plt.xlabel("Max Depth")
    plt.ylabel("Accuracy")
    plt.legend(loc='upper right')
    plt.show()

# Create two empty lists for train and test accuracy then iterate over a list of depths
train_accuracy = []
test_accuracy = []

depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]

for d in depths:
    dtc = DecisionTreeClassifier(max_depth=d)
    dtc.fit(X_train, y_train)
    train_accuracy.append(dtc.score(X_train,y_train))
    test_accuracy.append(dtc.score(X_test,y_test))
    
plot_train_test_accuracy(depths, train_accuracy, test_accuracy)


### Assessing DTC Performance

Unlike a KNN, where the model is most complex where `k` equals `1`, in a DTC, the deeper the tree (i.e. the larger `max_depth` is), the more complex the model. This is sharply evidenced in the performance over a range of `max_depth` values: performance on the training set and test set diverges as the DTC gets deeper. 

This means that the deeper the tree, the higher the variance. In the plot above, where is the point of highest bias? What would happen if we kept increasing `max_depth`?

How does this model compare to KNNs? Before introducing the DTC model, we noted that KNNs treat Mondays (`1`) as maximally different from Sundays (`7`) and queried whether or not this was a drawback of the model. Do DTCs suffer from the same limitation? What happens if the training data changes?

